In [1]:
from ModelResult import ModelResult

First, define a few functions which we will use to consistently apply what the changes in carbon and climate will be in computing $\beta,\gamma$.

In [2]:
def CarbonChange(v):
    v = v.integrateInSpace()
    v = v.accumulateInTime().convert("Pg")
    return v.data[-1]-v.data[0]

def ClimateChange(v):
    v = v.integrateInSpace(mean=True)
    return v.data[-1]-v.data[0]

Setup a model group for CanESM5 where the groups are based on which `1pctCO2` variant present in the filename.

In [3]:
m = ModelResult("/home/nate/data/ILAMB/MODELS/1pctCO2/CanESM5",name="CanESM5")
m.findFiles(group_regex=".*_(.*)_r1i1p1f1*")
print(m)

ModelResult: CanESM5
--------------------
  + 1pctCO2
  + 1pctCO2-bgc
  + 1pctCO2-rad



Extract the variables we will need. In this case we do not need a multimodel mean and so we leave the keyword `mean` to its default `False`.

In [4]:
nbp   = m.getVariable(  "nbp")
fgco2 = m.getVariable("fgco2")
tas   = m.getVariable(  "tas")

Now we use our functions to compute the changes in carbon and climate and store these in dictionaries where the keys will be the `1pctCO2` variant which corresponds to the data. `dL` is for the change in carbon over land, `dO` is for the change over ocean, and `dT` is the change in temperature.

In [5]:
dL = {}; dO = {}; dT = {}
for key in   nbp: dL[key] = CarbonChange (  nbp[key])
for key in fgco2: dO[key] = CarbonChange (fgco2[key])
for key in   tas: dT[key] = ClimateChange(  tas[key])

$\beta$ and $\gamma$ are defined using equations (2-5) of (Friedlingstein 2006)

$$
\begin{align*}
\Delta C_L^c &= \beta_L \Delta C_A^c + \gamma_L \Delta T^c\\
\Delta C_O^c &= \beta_O \Delta C_A^c + \gamma_O \Delta T^c\\
\Delta C_L^u &= \beta_L \Delta C_A^u\\
\Delta C_O^u &= \beta_O \Delta C_A^u
\end{align*}
$$

where the suberscripts $^{c}$ and $^{u}$ refer to *coupled* and *uncoupled*, subscripts $_A$, $_L$, and $_O$ refer to atmosphere, land, and ocean respectively. Now we just need to relate the model outputs to each variable:

* $^c$ - I think refers to the `1pctCO2` output
* $^u$ - I am not sure? In what sense should this be uncoupled?
* $\Delta C_L$ - This is the change in accumulative nbp, `dL`
* $\Delta C_O$ - This is the change in accumulative fgco2, `dO`

Then there is the question of how the atmopheric change in carbon is computed. If I understand what a 1% run is ($r=0.01$ below), the carbon in year 1 relative to year 0 should be:

$$
C_1 = C_0 (1+r)
$$

So the change in carbon after $n$ years is:

$$
\Delta C = C_0 ((1+r)^n-1)
$$

If $C_0=285$ [ppm] and we run from 1850 through 2000, then

$$
\Delta C = 285 (1.01^{151}-1) = 995.47
$$

Does this seem right? Is this the coupled or uncoupled number? What do I do for the other?